In [2]:
pip install sklearn_crfsuite

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 15.7 MB/s eta 0:00:00


In [3]:
pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 20.0 MB/s eta 0:00:00


In [4]:
from sklearn_crfsuite import CRF
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder
from sklearn.metrics import classification_report
from sklearn.utils import class_weight
import random
import itertools
import numpy as np
import scipy
import pandas as pd
from sklearn import metrics
from sklearn_crfsuite.utils import flatten
from datasets import load_dataset
import warnings
warnings.filterwarnings('ignore')

### Load Dataset

In [5]:
dataset = load_dataset("conll2003")
train_set = dataset["train"]
validation_set = dataset["validation"]
test_set = dataset["test"]

Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

Dataset conll2003 downloaded and prepared to /root/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
# A function to convert digit labels to string labels
def convert_ner_tags(dataset):
  # Label mapping from integer to string
  label_map = {0: 'O', 1: 'PER', 2: 'B-PER', 3: 'I-PER', 4: 'B-ORG', 5: 'I-ORG', 6: 'B-LOC', 7: 'I-LOC', 8: 'B-MISC', 9: 'I-MISC'}
  list_sent = []
  list_labels = []
  for sent in dataset:
    words = []
    label = []
    for word, label_idx in zip(sent['tokens'], sent['ner_tags']):
        label.append(label_map[label_idx])
        words.append(word)
    list_sent.append(words)
    list_labels.append(label)
  return list_sent, list_labels

In [7]:
# Define the train, validation and test set
train_ner, train_label_ner = convert_ner_tags(train_set)
validation_ner, validation_label_ner  = convert_ner_tags(validation_set)
test_ner, test_label_ner = convert_ner_tags(test_set)

In [8]:
print('Train',len(train_ner),len(train_label_ner))
print('Test',len(test_ner),len(test_label_ner))
print('Validation',len(validation_ner),len(validation_label_ner))

Train 14041 14041
Test 3453 3453
Validation 3250 3250


In [9]:
# Increase size of train set
new_train = []
new_train.extend(train_ner)
new_train.extend(validation_ner)

new_label = []
new_label.extend(train_label_ner)
new_label.extend(validation_label_ner)

print('Full',len(new_train),len(new_label))

Full 17291 17291


In [10]:
# A function to keep the digit labels
def ner_tags(dataset):
  list_sent = []
  list_labels = []
  for sent in dataset:
    words = []
    label = []
    for word, label_idx in zip(sent['tokens'], sent['ner_tags']):
        label.append(label_idx)
        words.append(word)
    list_sent.append(words)
    list_labels.append(label)
  return list_sent, list_labels

train_ner_digit, train_label_ner_digit = ner_tags(train_set)

### Weights of Classes

In [11]:
# Compute class weights
from sklearn.utils.class_weight import compute_class_weight

def modify_data(X, y):
  seq_x = []
  seq_y = []
  for sequence, labels in zip(X, y):
    modified_X = []
    modified_y = []
    for word, label in zip(sequence,labels):
      # Check if the current label is from the minority class
      if label != "O":
        modified_X.append(word)
        modified_y.append(label)

      else:
        # Determine the number of times to replicate the sequence based on class weights
        rr = random.uniform(0, 1)
        if rr > 0.6:
        # Replicate the sequence and label
          modified_X.append(word)
          modified_y.append(label)

    seq_x.append(modified_X)
    seq_y.append(modified_y)

  return seq_x, seq_y


class_weights = compute_class_weight(class_weight = "balanced", classes= np.unique(flatten(train_label_ner)), y= flatten(train_label_ner))

# Convert the class weights to a dictionary
class_weights_dict = dict(zip(np.unique(flatten(train_label_ner)), class_weights))

modified_X, modified_y = modify_data(train_ner, train_label_ner)

print(class_weights_dict)

{'B-LOC': 19.554499183712668, 'B-MISC': 19.58835978835979, 'B-ORG': 6.108141348692104, 'B-PER': 4.996589124460149, 'I-LOC': 6.580731691551936, 'I-ORG': 3.1687052598817305, 'I-PER': 3.5792683998664065, 'O': 0.13341680852206983, 'PER': 3.427962962962963}


### CRF Model Train

In [18]:
# Initialize the CRF model with class weighting
crf = CRF(algorithm='lbfgs', c1=0.25, c2=0.03, max_iterations=400, all_possible_states=True)

# Fit the CRF model on the modified training data
crf.fit(modified_X, modified_y)
print("done")

done


### Evaluation

In [19]:
y_pred = crf.predict(test_ner)
print(test_label_ner[0])
print(y_pred[0])

['O', 'O', 'I-ORG', 'O', 'O', 'O', 'O', 'PER', 'O', 'O', 'O', 'O']
['I-PER', 'O', 'I-LOC', 'B-MISC', 'B-MISC', 'B-MISC', 'O', 'I-PER', 'B-ORG', 'I-PER', 'B-ORG', 'O']


In [20]:
# Calculate evaluation metrics
report = classification_report(flatten(test_label_ner), flatten(y_pred), digits=3)
print(report)

              precision    recall  f1-score   support

       B-LOC      0.299     0.167     0.214       257
      B-MISC      0.203     0.282     0.236       216
       B-ORG      0.271     0.223     0.245       835
       B-PER      0.538     0.599     0.567      1156
       I-LOC      0.268     0.138     0.182       702
       I-ORG      0.377     0.428     0.401      1668
       I-PER      0.340     0.259     0.294      1661
           O      0.934     0.952     0.943     38323
         PER      0.500     0.443     0.470      1617

    accuracy                          0.849     46435
   macro avg      0.415     0.388     0.395     46435
weighted avg      0.839     0.849     0.843     46435

